<a href="https://colab.research.google.com/github/darshanraool/EV_Range_Predictor/blob/master/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import os
from IPython.display import display
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [31]:
data = '/content/drive/MyDrive/volkswagen_e_golf.csv'
df = pd.read_csv(data,encoding="ISO-8859-1")

In [32]:
# Reading data and removing unnecessary column
def remove_feature(data):
  new_df = data.drop(['manufacturer','model','version','fuel_date','fuel_type'],axis=1)
  if 'fuel_note' in new_df.columns:
    # Remove the specified column
    new_df.drop('fuel_note',axis=1, inplace=True)
  return new_df

ev_data = remove_feature(df)

In [33]:
ev_data.dtypes

power(kW)                   int64
trip_distance(km)          object
quantity(kWh)             float64
tire_type                  object
city                        int64
motor_way                   int64
country_roads               int64
driving_style              object
consumption(kWh/100km)    float64
A/C                         int64
park_heating                int64
avg_speed(km/h)           float64
ecr_deviation             float64
dtype: object

In [34]:
# Changing datatypes of columns
ev_data['city'] = ev_data['city'].astype('object')
ev_data['motor_way'] = ev_data['motor_way'].astype('object')
ev_data['country_roads'] = ev_data['country_roads'].astype('object')
ev_data['A/C'] = ev_data['A/C'].astype('object')
ev_data['park_heating'] = ev_data['park_heating']
if ev_data['trip_distance(km)'].dtypes == 'object':
  ev_data['trip_distance(km)']= ev_data['trip_distance(km)'].str.split(",").str[0]
ev_data['trip_distance(km)']= ev_data['trip_distance(km)'].astype('float')

In [35]:
# Removing Trip Distance Rows with NULL values
ev_data = ev_data[ev_data['trip_distance(km)'].isnull() == False]

In [36]:
# Get columns with null values
null_columns = ev_data.columns[ev_data.isnull().any()]

# Create a dictionary to store column names and their corresponding null percentages
null_percentages = {}

# Calculate null percentages
for column in null_columns:
    null_percentage = (ev_data[column].isnull().sum() / len(ev_data)) * 100
    null_percentages[column] = null_percentage

# Convert the dictionary to a DataFrame
null_df = pd.DataFrame.from_dict(null_percentages, orient='index', columns=['Null Percentage'])

print("Columns with null values and their corresponding null percentages:")
print(null_df)


Columns with null values and their corresponding null percentages:
                 Null Percentage
quantity(kWh)           0.029904
avg_speed(km/h)         0.358852


In [37]:
threshold = 30
columns_to_drop = null_df[null_df['Null Percentage'] > threshold].index

# Drop columns from the original DataFrame
df_cleaned = ev_data.drop(columns=columns_to_drop)
df_cleaned

,power(kW),trip_distance(km),quantity(kWh),tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation
0,85,80.0,NaN,Winter tires,0,0,1,Normal,17.8,0,1,53.0,1.0
1,85,50.0,12.29,Winter tires,0,0,1,Normal,15.5,0,1,47.0,-1.3
2,85,43.0,8.68,Winter tires,0,1,1,Normal,18.0,0,1,58.0,1.2
3,85,44.0,1.50,Winter tires,0,1,1,Normal,16.1,0,1,43.0,-0.7
4,85,76.0,14.44,Winter tires,0,1,0,Normal,19.0,0,1,76.0,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3340,85,26.0,5.62,Winter tires,1,1,1,Normal,21.6,1,0,45.0,4.8
3341,85,77.0,14.25,Winter tires,1,1,1,Normal,18.5,1,0,42.0,1.7
3342,85,26.0,4.97,Winter tires,1,1,1,Normal,19.1,1,0,23.0,2.3
3343,85,20.0,4.76,Winter tires,1,1,1,Fast,23.8,1,0,46.0,7.0


In [38]:
df_cleaned.isnull().sum()

power(kW)                  0
trip_distance(km)          0
quantity(kWh)              1
tire_type                  0
city                       0
motor_way                  0
country_roads              0
driving_style              0
consumption(kWh/100km)     0
A/C                        0
park_heating               0
avg_speed(km/h)           12
ecr_deviation              0
dtype: int64

In [39]:
categorical_columns = [col for col in df_cleaned.columns if pd.api.types.is_categorical_dtype(df_cleaned[col]) or df_cleaned[col].dtype == 'object']

# Fill NaN values in categorical columns with the majority (mode) value
for column in categorical_columns:
    mode_value = df_cleaned[column].mode()[0]  # Get the mode value of the column
    df_cleaned[column].fillna(mode_value, inplace=True)

print("DataFrame after filling NaN values with majority (mode) values in categorical columns:")
df_cleaned

DataFrame after filling NaN values with majority (mode) values in categorical columns:


,power(kW),trip_distance(km),quantity(kWh),tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation
0,85,80.0,NaN,Winter tires,0,0,1,Normal,17.8,0,1,53.0,1.0
1,85,50.0,12.29,Winter tires,0,0,1,Normal,15.5,0,1,47.0,-1.3
2,85,43.0,8.68,Winter tires,0,1,1,Normal,18.0,0,1,58.0,1.2
3,85,44.0,1.50,Winter tires,0,1,1,Normal,16.1,0,1,43.0,-0.7
4,85,76.0,14.44,Winter tires,0,1,0,Normal,19.0,0,1,76.0,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3340,85,26.0,5.62,Winter tires,1,1,1,Normal,21.6,1,0,45.0,4.8
3341,85,77.0,14.25,Winter tires,1,1,1,Normal,18.5,1,0,42.0,1.7
3342,85,26.0,4.97,Winter tires,1,1,1,Normal,19.1,1,0,23.0,2.3
3343,85,20.0,4.76,Winter tires,1,1,1,Fast,23.8,1,0,46.0,7.0


In [40]:
numerical_columns = df_cleaned.select_dtypes(include=['int64','float64']).columns

# Create the SimpleImputer with strategy='mean'
imputer = SimpleImputer(strategy='mean')

# Fit and transform the imputer on the numerical columns
df_cleaned[numerical_columns] = imputer.fit_transform(df_cleaned[numerical_columns])

print("DataFrame after filling NaN values with mean in numerical columns:")
df_cleaned

DataFrame after filling NaN values with mean in numerical columns:


,power(kW),trip_distance(km),quantity(kWh),tire_type,city,motor_way,country_roads,driving_style,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation
0,85.0,80.0,6.13945,Winter tires,0,0,1,Normal,17.8,0,1.0,53.0,1.0
1,85.0,50.0,12.29000,Winter tires,0,0,1,Normal,15.5,0,1.0,47.0,-1.3
2,85.0,43.0,8.68000,Winter tires,0,1,1,Normal,18.0,0,1.0,58.0,1.2
3,85.0,44.0,1.50000,Winter tires,0,1,1,Normal,16.1,0,1.0,43.0,-0.7
4,85.0,76.0,14.44000,Winter tires,0,1,0,Normal,19.0,0,1.0,76.0,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3340,85.0,26.0,5.62000,Winter tires,1,1,1,Normal,21.6,1,0.0,45.0,4.8
3341,85.0,77.0,14.25000,Winter tires,1,1,1,Normal,18.5,1,0.0,42.0,1.7
3342,85.0,26.0,4.97000,Winter tires,1,1,1,Normal,19.1,1,0.0,23.0,2.3
3343,85.0,20.0,4.76000,Winter tires,1,1,1,Fast,23.8,1,0.0,46.0,7.0


In [41]:
df_cleaned.isnull().sum()

power(kW)                 0
trip_distance(km)         0
quantity(kWh)             0
tire_type                 0
city                      0
motor_way                 0
country_roads             0
driving_style             0
consumption(kWh/100km)    0
A/C                       0
park_heating              0
avg_speed(km/h)           0
ecr_deviation             0
dtype: int64

In [42]:
outlier_columns=df_cleaned.select_dtypes(exclude='object')

In [43]:
def count_outliers(data,column):
    count=0
    q1=data[column].describe()[4]
    q3=data[column].describe()[6]
    iqr=q3-q1
    for i in data[column]:
        if (i<q1-(1.5*iqr)) or (i>q3+(1.5*iqr)):
            count+=1
    return count

In [44]:
for column in outlier_columns:
    print("No of outliers in {} are {}".format(column,count_outliers(df_cleaned,column)))

No of outliers in power(kW) are 0
No of outliers in trip_distance(km) are 154
No of outliers in quantity(kWh) are 155
No of outliers in consumption(kWh/100km) are 20
No of outliers in park_heating are 631
No of outliers in avg_speed(km/h) are 32
No of outliers in ecr_deviation are 20


In [45]:
def remove_outliers(data,column_list):
    for column in column_list:
        q1=data[column].describe()[4]
        q3=data[column].describe()[6]
        iqr=q3-q1
        for i in data[column]:
            if (i<q1-(1.5*iqr)) or (i>q3+(1.5*iqr)):
                data = data.loc[data[column] != i]
    return data

In [46]:
data=remove_outliers(df_cleaned,outlier_columns)

In [47]:
data.shape

(2430, 13)

In [48]:
ordinalEncoder = OrdinalEncoder(categories=[['Normal', 'Moderate', 'Fast']])
data['encoded_driving_style'] = ordinalEncoder.fit_transform(data.driving_style.values.reshape(-1,1))
data.drop("driving_style",axis=1,inplace=True)

labelEncoder = LabelEncoder()
data['encoded_tire_type'] = labelEncoder.fit_transform(data.tire_type)
data.drop("tire_type",axis=1,inplace=True)

In [49]:
data.head()

,power(kW),trip_distance(km),quantity(kWh),city,motor_way,country_roads,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation,encoded_driving_style,encoded_tire_type
10,85.0,80.0,13.08,0,0,1,14.0,0,0.0,52.0,-2.8,1.0,1
11,85.0,59.0,4.61,0,0,1,14.6,0,0.0,66.0,-2.2,1.0,1
21,85.0,60.0,3.00,0,0,1,13.5,0,0.0,57.0,-3.3,1.0,1
39,85.0,44.0,9.68,0,0,1,16.4,0,0.0,17.0,-0.4,0.0,1
40,85.0,78.0,8.91,0,0,1,16.0,0,0.0,51.0,-0.8,0.0,1


In [50]:
data[['city','motor_way','country_roads','A/C']] = pd.get_dummies(data[['city','motor_way','country_roads','A/C']],drop_first=True)
data.columns

<ipython-input-50-2346ad3cfc89>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data[['city','motor_way','country_roads','A/C']] = pd.get_dummies(data[['city','motor_way','country_roads','A/C']],drop_first=True)
<ipython-input-50-2346ad3cfc89>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data[['city','motor_way','country_roads','A/C']] = pd.get_dummies(data[['city','motor_way','country_roads','A/C']],drop_first=True)
<ipython-input-50-2346ad3cfc89>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data[['city','motor_way','country_roads','A/C']] = pd.get_dummies(data[['city','motor_way','country_roads','A/C']],drop_first=True)
<ipython-input-50-2346ad3cfc89>:1: FutureWa

Index(['power(kW)', 'trip_distance(km)', 'quantity(kWh)', 'city', 'motor_way',
       'country_roads', 'consumption(kWh/100km)', 'A/C', 'park_heating',
       'avg_speed(km/h)', 'ecr_deviation', 'encoded_driving_style',
       'encoded_tire_type'],
      dtype='object')

In [51]:
data

,power(kW),trip_distance(km),quantity(kWh),city,motor_way,country_roads,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation,encoded_driving_style,encoded_tire_type
10,85.0,80.0,13.08,0,0,1,14.0,0,0.0,52.0,-2.8,1.0,1
11,85.0,59.0,4.61,0,0,1,14.6,0,0.0,66.0,-2.2,1.0,1
21,85.0,60.0,3.00,0,0,1,13.5,0,0.0,57.0,-3.3,1.0,1
39,85.0,44.0,9.68,0,0,1,16.4,0,0.0,17.0,-0.4,0.0,1
40,85.0,78.0,8.91,0,0,1,16.0,0,0.0,51.0,-0.8,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,85.0,18.0,3.56,1,1,1,19.8,1,0.0,31.0,3.0,0.0,1
3340,85.0,26.0,5.62,1,1,1,21.6,1,0.0,45.0,4.8,0.0,1
3342,85.0,26.0,4.97,1,1,1,19.1,1,0.0,23.0,2.3,0.0,1
3343,85.0,20.0,4.76,1,1,1,23.8,1,0.0,46.0,7.0,2.0,1


In [52]:
data.columns

Index(['power(kW)', 'trip_distance(km)', 'quantity(kWh)', 'city', 'motor_way',
       'country_roads', 'consumption(kWh/100km)', 'A/C', 'park_heating',
       'avg_speed(km/h)', 'ecr_deviation', 'encoded_driving_style',
       'encoded_tire_type'],
      dtype='object')

In [53]:
data.dtypes

power(kW)                 float64
trip_distance(km)         float64
quantity(kWh)             float64
city                        uint8
motor_way                   uint8
country_roads               uint8
consumption(kWh/100km)    float64
A/C                         uint8
park_heating              float64
avg_speed(km/h)           float64
ecr_deviation             float64
encoded_driving_style     float64
encoded_tire_type           int64
dtype: object

In [54]:
data.to_csv("Cleaned Data.csv",index=False)

In [55]:
data.head()

,power(kW),trip_distance(km),quantity(kWh),city,motor_way,country_roads,consumption(kWh/100km),A/C,park_heating,avg_speed(km/h),ecr_deviation,encoded_driving_style,encoded_tire_type
10,85.0,80.0,13.08,0,0,1,14.0,0,0.0,52.0,-2.8,1.0,1
11,85.0,59.0,4.61,0,0,1,14.6,0,0.0,66.0,-2.2,1.0,1
21,85.0,60.0,3.00,0,0,1,13.5,0,0.0,57.0,-3.3,1.0,1
39,85.0,44.0,9.68,0,0,1,16.4,0,0.0,17.0,-0.4,0.0,1
40,85.0,78.0,8.91,0,0,1,16.0,0,0.0,51.0,-0.8,0.0,1
